In [2]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano' # Why theano why not
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

Using Theano backend.


In [3]:
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [4]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [22]:
df = pd.read_csv('/Users/3arthzneiz/Downloads/sentiment-analysis-on-movie-reviews/train.tsv', sep='\t')
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df['Sentiment'])))

Shape of dataset  (156060, 4)
Index(['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], dtype='object')
No. of unique classes 5


In [15]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [17]:
# digitize
macronum=sorted(set(df['Sentiment']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df['class']=df['class'].apply(fun)

In [23]:
texts = []
labels = []


for idx in range(df.Phrase.shape[0]):
    text = BeautifulSoup(df.Phrase[idx])
    texts.append(clean_str(str(text.get_text().encode())))

for idx in df['Sentiment']:
    labels.append(idx)

/Users/3arthzneiz/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/3arthzneiz/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/Users/3arthzneiz/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


In [24]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

In [30]:
sequences = tokenizer.texts_to_sequences(texts)

In [31]:
sequences

[[12,
  326,
  3,
  19058,
  8679,
  1,
  6889,
  9,
  70,
  11,
  52,
  15,
  1,
  3244,
  11,
  198,
  52,
  15,
  1,
  12027,
  80,
  3,
  84,
  818,
  12028,
  20,
  906,
  3,
  84,
  2485,
  5,
  64,
  3,
  4,
  45],
 [12, 326, 3, 19058, 8679, 1, 6889, 9, 70, 11, 52, 15, 1, 3244],
 [12, 326],
 [12],
 [],
 [28, 19058, 8679, 1, 6889, 9, 70, 11, 52, 15, 1, 3244],
 [28],
 [8679, 1, 6889, 9, 70, 11, 52, 15, 1, 3244],
 [],
 [13232, 1, 6889, 9, 70, 11, 52, 15, 1, 3244],
 [13232, 1, 6889],
 [13232],
 [8, 6889],
 [8],
 [],
 [53, 70, 11, 52, 15, 1, 3244],
 [53],
 [260, 11, 52, 15, 1, 3244],
 [260],
 [22, 52, 15, 1, 3244],
 [22],
 [577, 15, 1, 3244],
 [577],
 [112, 1, 3244],
 [112],
 [8, 3244],
 [15857],
 [22,
  198,
  52,
  15,
  1,
  12027,
  80,
  3,
  84,
  818,
  12028,
  20,
  906,
  3,
  84,
  2485,
  5,
  64,
  3,
  4,
  45],
 [22,
  198,
  52,
  15,
  1,
  12027,
  80,
  3,
  84,
  818,
  12028,
  20,
  906,
  3,
  84,
  2485,
  5,
  64,
  3,
  4,
  45],
 [22, 198],
 [1854],
 [577,


In [28]:
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [38]:
labels = to_categorical(np.asarray(labels))
print('Shape of Data Tensor:', data.shape)
print('Shape of Label Tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of Data Tensor: (156060, 1000)
Shape of Label Tensor: (156060, 5)


In [39]:
embeddings_index = {}
f = open('/Volumes/earth/glove.6B/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [42]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [44]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(len(macronum), activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

Simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2988500   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 

In [ ]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=2,callbacks=[cp])

Train on 124848 samples, validate on 31212 samples
Epoch 1/15
124848/124848 [==============================] - 20936s 168ms/step - loss: 1.2888 - acc: 0.5502 - val_loss: 2.6997 - val_acc: 0.5649

Epoch 00001: val_acc improved from -inf to 0.56491, saving model to model_cnn.hdf5
Epoch 2/15
124848/124848 [==============================] - 23096s 185ms/step - loss: 1.5914 - acc: 0.5662 - val_loss: 2.9231 - val_acc: 0.5514

Epoch 00002: val_acc did not improve from 0.56491
Epoch 3/15
  2742/124848 [..............................] - ETA: 5:14:29 - loss: 1.7270 - acc: 0.5460